In [18]:
# !pip install boto3==1.15.18
# !pip install gluonnlp==0.10.0
# !pip install onnxruntime==1.8.0
# !pip install sentencepiece==0.1.96
# !pip install torch==1.10.1+cu102 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers==4.8.1
# !pip install ipywidgets
# !pip install pandas
# !pip install scikit-learn
# !pip install torch
# !pip install transformers
# #인스톨이 안될경우 관리자 권한으로 vscode 실행바람


# !pip install --upgrade transformers
# #transformers 라이브러리 업그레이드

# !pip install kobert-transformers
# #kobert transformers 라이브러리 인스톨

In [1]:
import json

with open('../data/mentor_mentee_data.json', 'r', encoding='utf-8') as f:
    data_list = json.load(f)

In [ ]:
import pandas as pd

texts = []  # 멘토와 멘티 소개를 결합한 텍스트 데이터를 저장할 리스트
labels = []  # 멘토-멘티의 매칭 여부(라벨)를 저장할 리스트

# 주어진 데이터 리스트(data_list)에서 멘토와 멘티의 정보를 추출
for pair in data_list:
    mentor_bio = pair['mentor']['bio']  # 멘토의 소개 정보
    mentee_bio = pair['mentee']['bio']  # 멘티의 소개 정보
    match = pair['match']  # 멘토-멘티의 매칭 여부 (라벨 값, 예: 1 또는 0)

    # 멘토와 멘티의 소개를 결합하여 하나의 텍스트로 생성
    text = f"멘토 소개: {mentor_bio} 멘티 소개: {mentee_bio}"
    
    # 텍스트 데이터를 리스트에 추가
    texts.append(text)
    
    # 매칭 여부를 라벨 리스트에 추가
    labels.append(match)

# texts와 labels 리스트를 이용해 데이터프레임 생성
df = pd.DataFrame({'text': texts, 'label': labels})


In [4]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'], test_size=0.3, random_state=42)


In [5]:
from transformers import ElectraTokenizer, ElectraModel

# KoELECTRA 모델과 토크나이저 로드
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
model = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator')



c:\Users\smhrd1\Desktop\Mentors\ai\mentors_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from torch.utils.data import Dataset, DataLoader
import torch

class MentorMenteeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        """
        MentorMenteeDataset 클래스 초기화
        :param texts: 입력 텍스트 리스트 (멘토-멘티 소개)
        :param labels: 각 텍스트에 대한 라벨 리스트 (매칭 여부)
        :param tokenizer: 텍스트를 토큰화할 때 사용할 토크나이저 (예: KoBERT, KoELECTRA 등)
        :param max_len: 최대 토큰 길이 (문장을 토큰화할 때 자를 길이)
        """
        self.texts = texts.tolist()  # 텍스트 데이터를 리스트로 변환
        self.labels = labels.tolist()  # 라벨 데이터를 리스트로 변환
        self.tokenizer = tokenizer  # 토크나이저 객체 (BERT 기반)
        self.max_len = max_len  # 최대 토큰 길이
    
    def __len__(self):
        """
        데이터셋의 총 샘플 수를 반환
        :return: 총 샘플 수 (texts 리스트의 길이)
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 샘플을 반환
        :param idx: 데이터셋에서 가져올 샘플의 인덱스
        :return: 토큰화된 입력 데이터와 라벨 (딕셔너리 형태)
        """
        text = str(self.texts[idx])  # 인덱스에 해당하는 텍스트를 문자열로 변환
        label = self.labels[idx]  # 인덱스에 해당하는 라벨을 가져옴

        # 텍스트를 토큰화하고, 필요한 입력 형식으로 변환
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # [CLS], [SEP] 같은 특별 토큰 추가
            max_length=self.max_len,  # 최대 길이를 지정하여 텍스트 자름
            truncation=True,  # max_len을 초과하는 텍스트는 잘라냄
            padding='max_length',  # max_len에 맞춰 패딩 추가
            return_tensors='pt'  # PyTorch 텐서 형식으로 변환
        )

        # 토큰화된 결과를 딕셔너리 형태로 반환
        return {
            'input_ids': encoding['input_ids'].flatten(),  # 입력 토큰 ID (1차원 텐서로 변환)
            'attention_mask': encoding['attention_mask'].flatten(),  # 패딩 여부를 나타내는 마스크 (1차원 텐서)
            'labels': torch.tensor(label, dtype=torch.float)  # 라벨 값을 PyTorch 텐서로 변환 (float 형식)
        }


In [7]:
MAX_LEN = 128  # 각 입력 문장의 최대 토큰 길이 설정 (128 토큰으로 자르거나 패딩)
BATCH_SIZE = 16  # 한 번에 처리할 데이터 샘플 수 (배치 크기)

# 학습용 데이터셋을 생성
# - train_texts: 학습용 텍스트 데이터 리스트
# - train_labels: 학습용 라벨 리스트
# - tokenizer: 텍스트를 토큰화하는 데 사용할 BERT 기반 토크나이저
# - MAX_LEN: 각 문장의 최대 토큰 길이 (128)
train_dataset = MentorMenteeDataset(train_texts, train_labels, tokenizer, MAX_LEN)

# 검증용 데이터셋을 생성
# - val_texts: 검증용 텍스트 데이터 리스트
# - val_labels: 검증용 라벨 리스트
# - tokenizer: 검증용 데이터에도 같은 토크나이저와 설정을 사용
# - MAX_LEN: 검증 데이터에도 동일하게 128 토큰으로 자르거나 패딩
val_dataset = MentorMenteeDataset(val_texts, val_labels, tokenizer, MAX_LEN)

# 학습용 DataLoader를 생성
# - train_dataset: 학습용 MentorMenteeDataset 객체
# - batch_size: BATCH_SIZE (한 번에 처리할 데이터 수)
# - shuffle=True: 학습 시 데이터 순서를 섞어서 모델에 전달 (더 나은 일반화 성능)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# 검증용 DataLoader를 생성
# - val_dataset: 검증용 MentorMenteeDataset 객체
# - batch_size: BATCH_SIZE (검증 시에도 동일한 배치 크기로 처리)
# - shuffle=False: 검증 데이터는 섞지 않음 (일관된 평가를 위해 순서를 유지)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


In [8]:
import torch.nn as nn

class KoELECTRAClassifier(nn.Module):
    def __init__(self, electra):
        """
        KoELECTRAClassifier 클래스 초기화
        :param electra: 사전 학습된 ELECTRA 모델 (KoELECTRA)
        """
        super(KoELECTRAClassifier, self).__init__()
        self.electra = electra  # ELECTRA 모델을 할당
        self.dropout = nn.Dropout(0.3)  # 과적합 방지를 위한 드롭아웃 레이어 (30% 드롭아웃)
        self.classifier = nn.Linear(electra.config.hidden_size, 1)  # ELECTRA 출력(hidden_size)을 1차원으로 변환하는 선형 레이어
    
    def forward(self, input_ids, attention_mask):
        """
        순전파 계산
        :param input_ids: 입력 토큰 ID들
        :param attention_mask: 패딩된 부분을 무시하기 위한 어텐션 마스크
        :return: 최종 출력 로짓 (분류 점수)
        """
        # ELECTRA 모델의 순전파 진행 (입력값을 처리하여 마지막 히든 스테이트 반환)
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # [CLS] 토큰에 해당하는 첫 번째 토큰의 히든 스테이트를 사용
        pooled_output = outputs.last_hidden_state[:, 0]
        
        # 드롭아웃을 적용하여 과적합 방지
        dropout_output = self.dropout(pooled_output)
        
        # 선형 레이어를 통해 최종 분류 값 계산 (로짓 출력)
        logits = self.classifier(dropout_output)
        
        return logits


In [9]:
import torch
print(torch.cuda.is_available())  # True이면 GPU 사용 가능
print(torch.cuda.device_count())  # 사용 가능한 GPU 개수 출력
print(torch.cuda.get_device_name(0))  # 첫 번째 GPU 이름 출력


True
1
NVIDIA GeForce RTX 3060


In [10]:
import torch

# GPU가 사용 가능하면 CUDA 장치를, 그렇지 않으면 CPU를 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# KoELECTRAClassifier 모델을 생성하고, 지정된 장치(GPU 또는 CPU)로 이동
model = KoELECTRAClassifier(model)  # KoELECTRA 기반 분류 모델
model = model.to(device)  # 모델을 선택된 장치(GPU/CPU)로 이동

# 이진 분류 문제에 사용되는 손실 함수 설정
# BCEWithLogitsLoss는 출력 로짓(logits)을 입력으로 받아 이진 분류에서 손실을 계산
loss_fn = nn.BCEWithLogitsLoss().to(device)  # 손실 함수도 GPU로 이동

# AdamW 옵티마이저 설정
# 학습률(lr)은 2e-5로 설정되며, 모델의 파라미터들이 옵티마이저에 전달됨
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)


In [29]:
def train_epoch(model, data_loader, loss_fn, optimizer, device):
    """
    한 에폭 동안 모델을 학습하는 함수
    :param model: 학습할 모델
    :param data_loader: 학습 데이터 로더 (배치 단위로 데이터를 공급)
    :param loss_fn: 손실 함수 (BCEWithLogitsLoss)
    :param optimizer: 옵티마이저 (AdamW)
    :param device: 학습을 실행할 장치 (GPU 또는 CPU)
    :return: 배치당 평균 학습 손실 값
    """
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0  # 총 손실 값을 저장할 변수 초기화

    # 데이터 로더를 통해 배치 단위로 학습 데이터 처리
    for data in data_loader:
        # 입력 데이터와 라벨을 장치(GPU/CPU)로 이동
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device).unsqueeze(1)  # 라벨 차원을 모델 출력과 맞추기 위해 unsqueeze

        # 모델 순전파 계산 (입력 데이터를 모델에 전달하여 예측 값 얻기)
        outputs = model(input_ids, attention_mask)
        
        # 예측 값과 실제 라벨 간의 손실 계산
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()  # 손실 값을 누적
        
        # 역전파를 위한 그래디언트 초기화
        optimizer.zero_grad()
        
        # 역전파로 그래디언트 계산
        loss.backward()
        
        # 옵티마이저로 모델 파라미터 업데이트
        optimizer.step()

    # 배치당 평균 손실 반환
    return total_loss / len(data_loader)

def eval_model(model, data_loader, loss_fn, device):
    """
    모델을 평가하는 함수 (검증 데이터에 대한 손실 계산)
    :param model: 평가할 모델
    :param data_loader: 검증 데이터 로더 (배치 단위로 데이터를 공급)
    :param loss_fn: 손실 함수 (BCEWithLogitsLoss)
    :param device: 평가를 실행할 장치 (GPU 또는 CPU)
    :return: 배치당 평균 검증 손실 값
    """
    model.eval()  # 모델을 평가 모드로 설정
    total_loss = 0  # 총 손실 값을 저장할 변수 초기화

    # 평가 시 그래디언트를 계산하지 않도록 설정 (메모리 절약)
    with torch.no_grad():
        # 데이터 로더를 통해 배치 단위로 검증 데이터 처리
        for data in data_loader:
            # 입력 데이터와 라벨을 장치(GPU/CPU)로 이동
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device).unsqueeze(1)  # 라벨 차원을 맞추기 위해 unsqueeze

            # 모델 순전파 계산 (입력 데이터를 모델에 전달하여 예측 값 얻기)
            outputs = model(input_ids, attention_mask)
            
            # 예측 값과 실제 라벨 간의 손실 계산
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()  # 손실 값을 누적

    # 배치당 평균 손실 반환
    return total_loss / len(data_loader)


In [30]:
EPOCHS = 3  # 학습할 총 에폭 수 (총 3번의 에폭 동안 학습 및 검증 반복)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 사용 가능한 장치 설정 (GPU 사용 가능 시 CUDA, 그렇지 않으면 CPU)

# 에폭 수만큼 학습을 반복
for epoch in range(EPOCHS):
    # 현재 진행 중인 에폭 번호 출력 (에폭은 1부터 시작)
    print(f'Epoch {epoch+1}/{EPOCHS}')
    
    # 한 에폭 동안 학습 데이터로 모델을 학습
    train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device)
    
    # 한 에폭 동안 검증 데이터로 모델을 평가
    val_loss = eval_model(model, val_loader, loss_fn, device)
    
    # 학습 손실과 검증 손실을 출력
    print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')


Epoch 1/3
Train Loss: 0.2869 | Val Loss: 0.1164
Epoch 2/3
Train Loss: 0.0585 | Val Loss: 0.0120
Epoch 3/3
Train Loss: 0.0210 | Val Loss: 0.0146


In [31]:
# 모델 평가라인
def get_predictions(model, data_loader, device):
    model.eval()
    predictions = []
    real_values = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            preds = torch.sigmoid(outputs)
            preds = preds.cpu().numpy()
            labels = labels.cpu().numpy()

            predictions.extend(preds)
            real_values.extend(labels)

    return predictions, real_values

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

preds, labels = get_predictions(model, val_loader, device)
preds = [1 if p >= 0.5 else 0 for p in preds]

accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds)
recall = recall_score(labels, preds)
f1 = f1_score(labels, preds)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


Accuracy: 0.9956
Precision: 0.9608
Recall: 1.0000
F1 Score: 0.9800


In [51]:
#모델저장
torch.save(model.state_dict(), 'KoELECTRA_mentor_mentee.pth')


In [52]:

from google.colab import drive
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import ElectraTokenizer, ElectraModel
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator')
model = KoELECTRAClassifier(bert_model)
model.load_state_dict(torch.load('KoELECTRA_mentor_mentee.pth', map_location=device))
model = model.to(device)



<ipython-input-52-8028b0281d6f>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('KoELECTRA_mentor_mentee.pth', map_location=device))


In [53]:
#멘토 멘티 바이오 가져와서 매칭
def predict_match(mentor_bio, mentee_bio):
    text = f"멘토 소개: {mentor_bio} 멘티 소개: {mentee_bio}"

    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        output = model(input_ids, attention_mask)
        prob = torch.sigmoid(output)
        return prob.item()

# 예시 사용
mentor_bio = "고급 단계의 엑셀 전문가입니다. 직장 생활의 노하우를 전수해드리겠습니다."
mentee_bio = "엑셀 활용법에 관심 있는 중급 멘티입니다. 많은 것을 배우고 싶습니다."

match_prob = predict_match(mentor_bio, mentee_bio)
print(f'매칭 확률: {match_prob:.4f}')


매칭 확률: 0.0021


In [61]:
# 각 구성요소 매치시켜서 측정하는 함수
def predict_match_from_data(pair):
    mentor_expertise = ", ".join(pair['mentor']['expertise'])
    mentee_interests = ", ".join(pair['mentee']['interests'])
    mentor_level = pair['mentor']['level']
    mentee_level = pair['mentee']['level']
    mentor_bio = pair['mentor']['bio']
    mentee_bio = pair['mentee']['bio']

    # 학습 시 사용한 형식대로 입력 텍스트를 구성합니다.
    text = (
        f"멘토 전문 분야: {mentor_expertise}, 멘토 레벨: {mentor_level}, "
        f"멘토 소개: {mentor_bio}, 멘티 관심사: {mentee_interests}, "
        f"멘티 레벨: {mentee_level}, 멘티 소개: {mentee_bio}"
    )

    # 토크나이징 및 인코딩
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # 모델 예측
    model.eval()
    with torch.no_grad():
        output = model(input_ids, attention_mask)
        prob = torch.sigmoid(output)
        return prob.item()

def find_best_match(mentee, mentors):
    best_match_prob = -1
    best_mentor = None

    # 각 멘토와 멘티의 매칭 확률을 계산하고 가장 높은 매칭 확률을 가진 멘토 선택
    for mentor in mentors:
        pair = {
            "mentor": mentor,
            "mentee": mentee
        }
        match_prob = predict_match_from_data(pair)

        print(f"멘토 {mentor['name']}의 매칭 확률: {match_prob:.4f}")

        if match_prob > best_match_prob:
            best_match_prob = match_prob
            best_mentor = mentor

    return best_mentor, best_match_prob

# 예시 멘티 데이터
mentee = {
    "name": "윤하은",
    "interests": [
        "데이터 분석",
        "Python 프로그래밍"
    ],
    "level": 1,
    "bio": "데이터 분석과 Python 프로그래밍에 관심이 많은 취업 준비생입니다. 실무 경험을 쌓고 싶습니다."
}

# 여러 명의 멘토 리스트 (10명)
mentors = [
    {
        "name": "김수영",
        "expertise": [
            "데이터 시각화",
            "Python",
            "AI"
        ],
        "level": 3,
        "bio": "데이터 시각화 및 AI 분야의 전문가입니다. Python을 통한 AI 모델 개발에 경험이 많습니다."
    },
    {
        "name": "박민수",
        "expertise": [
            "프론트엔드 개발",
            "React",
            "JavaScript"
        ],
        "level": 2,
        "bio": "프론트엔드 개발과 React에 대한 깊은 이해를 바탕으로 웹 개발 프로젝트를 성공적으로 이끌었습니다."
    },
    {
        "name": "이하연",
        "expertise": [
            "데이터베이스",
            "SQL",
            "데이터 엔지니어링"
        ],
        "level": 3,
        "bio": "SQL 및 데이터베이스 설계에 대한 풍부한 경험이 있으며, 데이터 엔지니어링 프로젝트를 다수 진행했습니다."
    },
    {
        "name": "정윤호",
        "expertise": [
            "백엔드 개발",
            "Node.js",
            "AWS"
        ],
        "level": 3,
        "bio": "백엔드 개발과 클라우드 서비스(AWS)에 대한 전문성을 보유하고 있으며, 서버 운영과 관련된 다양한 경험이 있습니다."
    },
    {
        "name": "한지원",
        "expertise": [
            "기계 학습",
            "TensorFlow",
            "Keras"
        ],
        "level": 2,
        "bio": "기계 학습 모델 개발과 관련된 프로젝트 경험이 풍부하며, TensorFlow와 Keras를 다루는 데 능숙합니다."
    },
    {
        "name": "서지호",
        "expertise": [
            "비즈니스 분석",
            "프로젝트 관리",
            "엑셀"
        ],
        "level": 2,
        "bio": "엑셀을 활용한 비즈니스 분석과 프로젝트 관리 경험이 있으며, 기업의 생산성 향상을 위한 컨설팅을 제공하고 있습니다."
    },
    {
        "name": "김나윤",
        "expertise": [
            "PPT 디자인",
            "프레젠테이션",
            "마케팅 전략"
        ],
        "level": 3,
        "bio": "프레젠테이션과 PPT 디자인에 대한 경험이 풍부하며, 마케팅 전략을 효과적으로 전달하는 방법을 지도합니다."
    },
    {
        "name": "장민준",
        "expertise": [
            "AI 연구",
            "자연어 처리",
            "딥러닝"
        ],
        "level": 3,
        "bio": "AI 연구와 자연어 처리 분야에서 다수의 프로젝트를 진행했으며, 딥러닝 모델 개발에 전문성을 가지고 있습니다."
    },
    {
        "name": "최혜진",
        "expertise": [
            "UX/UI 디자인",
            "사용자 경험",
            "그래픽 디자인"
        ],
        "level": 3,
        "bio": "UX/UI 디자인에 대한 깊은 이해와 다양한 사용자 경험 프로젝트를 진행했습니다. 그래픽 디자인에도 능숙합니다."
    },
    {
        "name": "이상혁",
        "expertise": [
            "모바일 앱 개발",
            "Android",
            "iOS"
        ],
        "level": 2,
        "bio": "모바일 앱 개발(Android, iOS)에서의 경험이 많으며, 다양한 모바일 프로젝트를 성공적으로 완수했습니다."
    }
]

# 멘티와 가장 매칭 확률이 높은 멘토 찾기
best_mentor, best_prob = find_best_match(mentee, mentors)
print(f"가장 매칭률이 높은 멘토: {best_mentor['name']} (매칭 확률: {best_prob:.4f})")


멘토 김수영의 매칭 확률: 0.1479
멘토 박민수의 매칭 확률: 0.4234
멘토 이하연의 매칭 확률: 0.1517
멘토 정윤호의 매칭 확률: 0.1789
멘토 한지원의 매칭 확률: 0.1046
멘토 서지호의 매칭 확률: 0.2796
멘토 김나윤의 매칭 확률: 0.0762
멘토 장민준의 매칭 확률: 0.0684
멘토 최혜진의 매칭 확률: 0.2572
멘토 이상혁의 매칭 확률: 0.4830
가장 매칭률이 높은 멘토: 이상혁 (매칭 확률: 0.4830)


In [60]:
def predict_match_from_data(pair):
    mentor_expertise = ", ".join(pair['mentor']['expertise'])
    mentee_interests = ", ".join(pair['mentee']['interests'])
    mentor_level = pair['mentor']['level']
    mentee_level = pair['mentee']['level']
    mentor_bio = pair['mentor']['bio']
    mentee_bio = pair['mentee']['bio']

    # 학습 시 사용한 형식대로 입력 텍스트를 구성합니다.
    text = (
        f"멘토 전문 분야: {mentor_expertise}, 멘토 레벨: {mentor_level}, "
        f"멘토 소개: {mentor_bio}, 멘티 관심사: {mentee_interests}, "
        f"멘티 레벨: {mentee_level}, 멘티 소개: {mentee_bio}"
    )

    # 토크나이징 및 인코딩
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # 모델 예측
    model.eval()
    with torch.no_grad():
        output = model(input_ids, attention_mask)
        prob = torch.sigmoid(output)
        return prob.item()

def find_best_match(mentee, mentors):
    best_match_prob = -1
    best_mentor = None

    # 각 멘토와 멘티의 매칭 확률을 계산하고 가장 높은 매칭 확률을 가진 멘토 선택
    for mentor in mentors:
        pair = {
            "mentor": mentor,
            "mentee": mentee
        }
        match_prob = predict_match_from_data(pair)

        print(f"멘토 {mentor['name']}의 매칭 확률: {match_prob:.4f}")

        if match_prob > best_match_prob:
            best_match_prob = match_prob
            best_mentor = mentor

    return best_mentor, best_match_prob

# 예시 멘티 데이터
mentee = {
    "name": "한지우",
    "interests": [
        "생산성 향상",
        "스피치 기술",
        "엑셀 활용법"
    ],
    "level": 1,
    "bio": "생산성 향상, 스피치 기술, 엑셀 활용법에 관심 있는 초급 멘티입니다. 많은 것을 배우고 싶습니다."
}

# 여러 명의 멘토 리스트 (10명)
mentors = [
    {
        "name": "송다은",
        "expertise": [
            "취업 및 이직",
            "스피치 발표",
            "생산성 툴"
        ],
        "level": 2,
        "bio": "중급 수준의 취업 및 이직, 스피치 발표, 생산성 툴 전문가입니다. 직장 생활의 노하우를 전수해드리겠습니다."
    },
    {
        "name": "이하연",
        "expertise": [
            "데이터베이스",
            "SQL",
            "데이터 엔지니어링"
        ],
        "level": 5,
        "bio": "SQL 및 데이터베이스 설계에 대한 풍부한 경험이 있으며, 데이터 엔지니어링 프로젝트를 다수 진행했습니다."
    },
    {
        "name": "정윤호",
        "expertise": [
            "백엔드 개발",
            "Node.js",
            "AWS"
        ],
        "level": 4,
        "bio": "백엔드 개발과 클라우드 서비스(AWS)에 대한 전문성을 보유하고 있으며, 서버 운영과 관련된 다양한 경험이 있습니다."
    },
    {
        "name": "한지원",
        "expertise": [
            "기계 학습",
            "TensorFlow",
            "Keras"
        ],
        "level": 3,
        "bio": "기계 학습 모델 개발과 관련된 프로젝트 경험이 풍부하며, TensorFlow와 Keras를 다루는 데 능숙합니다."
    },
    {
        "name": "서지호",
        "expertise": [
            "비즈니스 분석",
            "프로젝트 관리",
            "엑셀"
        ],
        "level": 2,
        "bio": "엑셀을 활용한 비즈니스 분석과 프로젝트 관리 경험이 있으며, 기업의 생산성 향상을 위한 컨설팅을 제공하고 있습니다."
    },
    {
        "name": "김나윤",
        "expertise": [
            "PPT 디자인",
            "프레젠테이션",
            "마케팅 전략"
        ],
        "level": 3,
        "bio": "프레젠테이션과 PPT 디자인에 대한 경험이 풍부하며, 마케팅 전략을 효과적으로 전달하는 방법을 지도합니다."
    },
    {
        "name": "장민준",
        "expertise": [
            "AI 연구",
            "자연어 처리",
            "딥러닝"
        ],
        "level": 5,
        "bio": "AI 연구와 자연어 처리 분야에서 다수의 프로젝트를 진행했으며, 딥러닝 모델 개발에 전문성을 가지고 있습니다."
    },
    {
        "name": "최혜진",
        "expertise": [
            "UX/UI 디자인",
            "사용자 경험",
            "그래픽 디자인"
        ],
        "level": 3,
        "bio": "UX/UI 디자인에 대한 깊은 이해와 다양한 사용자 경험 프로젝트를 진행했습니다. 그래픽 디자인에도 능숙합니다."
    },
    {
        "name": "이상혁",
        "expertise": [
            "모바일 앱 개발",
            "Android",
            "iOS"
        ],
        "level": 4,
        "bio": "모바일 앱 개발(Android, iOS)에서의 경험이 많으며, 다양한 모바일 프로젝트를 성공적으로 완수했습니다."
    }
]

# 멘티와 가장 매칭 확률이 높은 멘토 찾기
best_mentor, best_prob = find_best_match(mentee, mentors)
print(f"가장 매칭률이 높은 멘토: {best_mentor['name']} (매칭 확률: {best_prob:.4f})")



멘토 송다은의 매칭 확률: 0.9855
멘토 이하연의 매칭 확률: 0.2353
멘토 정윤호의 매칭 확률: 0.6287
멘토 한지원의 매칭 확률: 0.1220
멘토 서지호의 매칭 확률: 0.7310
멘토 김나윤의 매칭 확률: 0.1283
멘토 장민준의 매칭 확률: 0.5103
멘토 최혜진의 매칭 확률: 0.5307
멘토 이상혁의 매칭 확률: 0.7501
가장 매칭률이 높은 멘토: 송다은 (매칭 확률: 0.9855)
